# Dynamic Flux Balance Analysis (dFBA)

In [ ]:
# !pip install cobra

In [35]:
import cobra
from cobra.io import load_model

ecoli = load_model("textbook")
ecoli.reactions.EX_o2_e.lower_bound = -2

In [36]:
ecoli

Name,e_coli_core
Memory address,176f10f70
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba
Compartments,"cytosol, extracellular"


## michaelis-menten kinetics

$
v = \frac{v_{max} \cdot [S]}{K_m + [S]}
$

In [37]:
# define the function
def perform_dfba(
    model, 
    initial_conditions, 
    kinetic_params, 
    reaction_names,
    biomass_name_map,
    dt,
):
    updated_state = initial_conditions.copy()

    for substrate_id, reaction_id in reaction_names.items():
        Km, Vmax = kinetic_params[substrate_id]
        substrate_concentration = initial_conditions[substrate_id]
        
        # calculate michaelis-menten flux
        flux = Vmax * substrate_concentration / (Km + substrate_concentration)

        # use the flux to constrain fba
        model.reactions.get_by_id(reaction_id).lower_bound = -flux

    # solve fba under these constraints
    solution = model.optimize()

    # gather the results
    ## update biomass
    biomass_growth_rate = solution.fluxes[biomass_name_map[1]]
    current_biomass = updated_state[biomass_name_map[0]]
    updated_state[biomass_name_map[0]] += biomass_growth_rate * current_biomass * dt

    ## update substrates
    for substrate_id, reaction_id in reaction_names.items():
        flux = solution.fluxes[reaction_id]
        current_substrate_conc = updated_state[substrate_id]
        updated_state[substrate_id] += flux * current_biomass * dt

    return updated_state



# example
conditions1 = {
    'biomass': 0.1,  # initial biomass concentration
    'glucose': 20.0, # initial glucose concentrations
    'acetate': 0.0,  # initial acetate concentrations
}
kinetics1 = {
    'glucose': (0.5, 2),  # km and vmax for glucose
    'acetate': (0.5, 2),  # km and vmax for acetate
}
rxn_name_map = {
    'glucose': 'EX_glc__D_e',
    'acetate': 'EX_ac_e'
}
biomass_name_map = ('biomass', 'Biomass_Ecoli_core')

time_interval = 1

# call the dfba function and get results
dfba_results = perform_dfba(
    model=ecoli, 
    initial_conditions=conditions1, 
    kinetic_params=kinetics1,
    reaction_names=rxn_name_map,
    biomass_name_map=biomass_name_map,
    dt=time_interval
)
dfba_results

{'biomass': 0.10375190928516904,
 'glucose': 19.804878048780488,
 'acetate': 0.3224987405359767}